In [ ]:
import random
from collections import defaultdict
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import itertools
from sklearn.manifold import TSNE
from sknetwork.data import karate_club, painters, movie_actor
from sknetwork.clustering import Louvain, BiLouvain, modularity, bimodularity
from sknetwork.linalg import normalize
from sknetwork.utils import bipartite2undirected, membership_matrix
from sknetwork.visualization import svg_graph, svg_digraph, svg_bigraph
from sknetwork.hierarchy import LouvainHierarchy, BiLouvainHierarchy
from sknetwork.hierarchy import dasgupta_score, dasgupta_cost
from sklearn.cluster import MiniBatchKMeans, KMeans, FeatureAgglomeration, AffinityPropagation, MeanShift, SpectralClustering, AgglomerativeClustering, AgglomerativeClustering, DBSCAN, Birch
import math
import pickle
import graphviz
from xgboost import plot_tree
import csv
import pandas as pd
import os
import json
import gensim
import pickle
import numpy as np
from pprint import pprint
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from ggplot import *

from imblearn.over_sampling import SMOTE

from scipy.stats import chi2_contingency
from matplotlib import rc

import scipy

import pickle
import shap
import random

import requests
import json
import re
from os.path import isfile, join
from os import listdir
from bs4 import BeautifulSoup
from collections import Counter

from ggplot import *
from sklearn import metrics
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
    auc,
    average_precision_score,
    fbeta_score
)

from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
import networkx as nx
import networkx
import community
from yellowbrick.cluster import KElbowVisualizer
from community import community_louvain

import os
import csv
import json
import scipy
import random
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from ggplot import *
from sklearn import metrics
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import (
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
    confusion_matrix,
    silhouette_score,
    silhouette_samples,
    calinski_harabasz_score,
    davies_bouldin_score,
    roc_curve,
    matthews_corrcoef,
    auc,
    balanced_accuracy_score
)


In [ ]:
with open('drugId2numId_nod2vec.pkl', 'rb') as f:
    drugId2numId_nod2vec = pickle.load(f)
with open('diseaseId2numId_nod2vec.pkl', 'rb') as f:
    diseaseId2numId_nod2vec = pickle.load(f)
with open('target2name.pkl', 'rb') as f:
    target2name = pickle.load(f)   
with open('drug_cut2chem.pkl', 'rb') as f:
    drug_cut2chem = pickle.load(f)

In [ ]:
ChEMBL = pd.read_csv('CHEMBL27_find_negative.csv', sep=';')

In [ ]:
ChEMBL_ph4 = ChEMBL[ChEMBL['Molecule Max Phase']==4]

In [ ]:
chmble2drugname = pd.Series(ChEMBL_ph4['Molecule Name'].values, index=ChEMBL_ph4['Molecule ChEMBL ID']).to_dict()

In [ ]:
AllDisGene = pd.read_pickle('CTD_genes_diseases.pkl')

In [ ]:
df_entrez_geneid = pd.read_csv("entrez_geneid.csv").dropna(subset=['entrezgene'])
df_entrez =  df_entrez_geneid[['symbol','entrezgene']]
df_entrez.index = df_entrez_geneid['entrezgene']
df_entrez = df_entrez.drop(columns=['entrezgene'])
entrezgene_dic = df_entrez.to_dict()['symbol']

In [ ]:
df_entrez_geneid2 = pd.read_csv("selected_uniport_seq.csv")[['id','Gene names']]
df_entrez_geneid2.index = df_entrez_geneid2['id']
df_entrez_geneid2 = df_entrez_geneid2.drop(columns=['id'])
entrezgene_dic2 = df_entrez_geneid2.to_dict()['Gene names']

In [ ]:
disease2name= pd.Series('disease', index=list(diseaseId2numId_nod2vec.values())).to_dict()
drug2name = pd.Series('drug', index=list(drugId2numId_nod2vec.values())).to_dict()

In [ ]:
dict_id = {**drug2name, **disease2name, **target2name}
pd.DataFrame(dict_id.items(), columns=['id', 'type'])['type'].value_counts()

In [ ]:
with open('final_interactions/DTI.pkl', 'rb') as f:
    DTI = pickle.load(f).reset_index(drop=True)
    
DTI_tmp_index = DTI[['from','to']].drop_duplicates().index
DTI = DTI[DTI.index.isin(DTI_tmp_index)]

In [ ]:
negative_DTI = pd.read_csv('negative_int.csv',index_col=0)
negative_DTI ['from'] = negative_DTI['Chembl'].str.replace('CHEMBL', '').astype(int).map(drugId2numId_nod2vec)
negative_DTI

In [ ]:
with open('all_nod2vec.pkl', 'rb') as f:
    all_nod2vec = pickle.load(f)

In [ ]:
numId2drugId = pd.Series(drugId2numId_nod2vec.keys(), index=drugId2numId_nod2vec.values()).to_dict()

In [ ]:
df_dds = all_nod2vec[all_nod2vec['Type_Interaction']=='DDS']
df_dds['from'] = df_dds['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
df_dds['to'] = df_dds['to'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
df_dds[['from','to','weight']].to_csv('DDS_with_id.csv')

In [ ]:
df_pps = all_nod2vec[all_nod2vec['Type_Interaction']=='PPS']
df_pps['from'] = df_pps['from'].map(entrezgene_dic2)
df_pps['to'] = df_pps['to'].map(entrezgene_dic2)
df_pps[['from','to','weight']].to_csv('PPS_with_id.csv')

In [ ]:
with open('all_nod2vec_new.pkl', 'rb') as f:
    all_nod2vec_new = pickle.load(f)
all_nod2vec_new

In [ ]:
mapping = False
df = all_nod2vec.copy()

######    ######    ######    ######    ######    ######    ######    ######    ######    ######    ######    ######

df = df[df['weight']!=0]
df = df.drop(columns= ['Type_Interaction'])

os.chdir("nod2vec")
df.to_csv('edglist_list.edgelist', sep=' ', index=False, header=False)
os.chdir("KEEG_pathway_Embedding")

if mapping:
    # nod2vec (for drug)
    os.system(f'PYTHONHASHSEED=10 python2 node2vec/src/main.py --workers 8 --input edglist_list.edgelist --output dim100.emb --weighted --dimensions 100')

embeddings_all = pd.read_csv('dim100.emb', sep=' ', skiprows=[0], header=None, index_col=0)  
embeddings_all

In [ ]:
def PCA_label(X, y):
    X_pca = X.copy()
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(X_pca.values)
    
    X_pca['PCA-1'] = pca_result[:,0]
    X_pca['PCA-2'] = pca_result[:,1]
    X_pca['type'] = y
    
    plt = ggplot(X_pca, aes(x='PCA-1', y='PCA-2',  color='factor(type)') ) \
    + geom_point(size=100, alpha=0.8)
    
    print(plt)

In [ ]:
def round_decimals_up(number:float, decimals:int=2):
    """
    Returns a value rounded up to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.ceil(number)

    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

In [ ]:
embeddings_all_id = embeddings_all.copy()
embeddings_all_id['type']= embeddings_all.index.map(dict_id)

pca = PCA(n_components=3)
pca_result = pca.fit_transform(embeddings_all_id.drop(columns=['type']).dropna().values)


embeddings_all_id['PCA-1'] = pca_result[:,0]
embeddings_all_id['PCA-2'] = pca_result[:,1]
embeddings_all_id['PCA-3'] = pca_result[:,2]


chart = ggplot(embeddings_all_id.dropna(), aes(x='PCA-1', y='PCA-2',  color='factor(type)') ) \
    + geom_point(size=120, alpha=0.8) 

chart

In [ ]:
Alicia_gene_name = embeddings_all_id[embeddings_all_id['type']=='target'][['type']]
entrezgene_dic2[8536] = 'CAMK1'
entrezgene_dic2[5753] = 'PTK6'
entrezgene_dic2[57172] = 'CAMK1G'
entrezgene_dic2[23678] = 'SGK3'
entrezgene_dic2[3303] = 'HSPA1A'
Alicia_gene_name['gene name'] = Alicia_gene_name.index.map(entrezgene_dic2)
Alicia_gene_name['entrez gene'] = Alicia_gene_name.index
Alicia_gene_name[['entrez gene', 'gene name']].to_csv('gene_list.csv')

In [ ]:
def community_layout(g, partition):
    """
    Compute the layout for a modular graph.


    Arguments:
    ----------
    g -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    partition -- dict mapping int node -> int community
        graph partitions


    Returns:
    --------
    pos -- dict mapping int node -> (float x, float y)
        node positions

    """

    pos_communities = _position_communities(g, partition, scale=3.)

    pos_nodes = _position_nodes(g, partition, scale=1.)

    # combine positions
    pos = dict()
    for node in g.nodes():
        pos[node] = pos_communities[node] + pos_nodes[node]

    return pos

def _position_communities(g, partition, **kwargs):

    # create a weighted graph, in which each node corresponds to a community,
    # and each edge weight to the number of edges between communities
    between_community_edges = _find_between_community_edges(g, partition)

    communities = set(partition.values())
    hypergraph = nx.DiGraph()
    hypergraph.add_nodes_from(communities)
    for (ci, cj), edges in between_community_edges.items():
        hypergraph.add_edge(ci, cj, weight=len(edges))

    # find layout for communities
    pos_communities = nx.spring_layout(hypergraph, **kwargs)

    # set node positions to position of community
    pos = dict()
    for node, community in partition.items():
        pos[node] = pos_communities[community]

    return pos

def _find_between_community_edges(g, partition):

    edges = dict()

    for (ni, nj) in g.edges():
        ci = partition[ni]
        cj = partition[nj]

        if ci != cj:
            try:
                edges[(ci, cj)] += [(ni, nj)]
            except KeyError:
                edges[(ci, cj)] = [(ni, nj)]

    return edges

def _position_nodes(g, partition, **kwargs):
    """
    Positions nodes within communities.
    """

    communities = dict()
    for node, community in partition.items():
        try:
            communities[community] += [node]
        except KeyError:
            communities[community] = [node]

    pos = dict()
    for ci, nodes in communities.items():
        subgraph = g.subgraph(nodes)
        pos_subgraph = nx.spring_layout(subgraph, **kwargs)
        pos.update(pos_subgraph)

    return pos

In [ ]:
all_nod2vec_new = all_nod2vec[all_nod2vec['weight']>=0.00][['from','to','weight']]
all_nod2vec_new

In [ ]:
target2cluster = pd.Series(embeddings_all_id['type'].map({'drug':0,'target':1,'disease':2}).values, index= embeddings_all_id.index).to_dict()
target2cluster

In [ ]:
embeddings_all_id[embeddings_all_id['type'].isna()]
target2cluster[100507538] = 2
target2cluster[100507539 ]= 2

In [ ]:
color_dict = {0: "#E07F80", 1: "#3E7274", 2: "#F2B342"}

In [ ]:
# clusters PPS network
edgeList_pps = all_nod2vec_new.values.tolist()
G = networkx.Graph()
weights = []

for i in range(len(edgeList_pps)):
    G.add_edge(edgeList_pps[i][0], edgeList_pps[i][1], weight=edgeList_pps[i][2])
    weights.append(edgeList_pps[i][2])
    
A = networkx.adjacency_matrix(G).A

In [ ]:
map_col = list(embeddings_all_id['type'].map({'drug':0,'target':1,'disease':2}).map({0: "#E07F80", 1: "#3E7274", 2: "#F2B342"}))

In [ ]:
embeddings_all_id_1 = embeddings_all_id.copy()
embeddings_all_id_1['type1']= embeddings_all_id['type'].map({'drug':0,'target':1,'disease':2}).map({0: "#E07F80", 1: "#3E7274", 2: "#F2B342"})
embeddings_all_id_1

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
sns.set(style = "white")

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection = '3d')

x = embeddings_all_id['PCA-1']
y = embeddings_all_id['PCA-2']
z = embeddings_all_id['PCA-3']

cmap_dict = {0: "#E07F80", 1: "#3E7274", 2: "#F2B342"}
cmap = ListedColormap(list(cmap_dict.values()))

ax.set_xlabel(" ")
ax.set_ylabel(" ")
ax.set_zlabel(" ")

sc = ax.scatter(x, y, z, s=40,cmap=cmap, c=embeddings_all_id['type'].map({'drug':0,'target':1,'disease':2}), marker='o', alpha=1)

plt.show()


In [ ]:
# removing nodes that we do not have embedding vectors
new_DTI = DTI[DTI['to'].isin(list(embeddings_all.index))]
new_DTI = new_DTI[new_DTI['from'].isin(list(embeddings_all.index))]
new_DTI

In [ ]:
new_DTI['label'] = new_DTI[['S1','S2','S3','S4','S5','S6']].values.tolist()
selected_type = ['increases^expression','decreases^expression','decreases^reaction','increases^reaction', 'increases^activity','decreases^activity']
for i in range (0, len(new_DTI)):
    new_DTI['label'].iloc[i] = list(set(new_DTI['label'].iloc[i])& set(selected_type))
    
print (new_DTI['Type_Interaction'].value_counts())

new_DTI = new_DTI[['from','to','label']]

In [ ]:
Alicia_gene_name = new_DTI.copy()
Alicia_gene_name['gene name']= Alicia_gene_name['to'].map(entrezgene_dic2)
Alicia_gene_name['entrez gene'] = Alicia_gene_name['to']
Alicia_gene_name[['entrez gene','gene name']].drop_duplicates().to_csv('gene_list.csv')

In [ ]:
Alicia_gene_name['entrez gene'] = Alicia_gene_name.index
Alicia_gene_name[['entrez gene', 'gene name']].to_csv('gene_list.csv')

In [ ]:
def concating(df, embedding):
    """
    This function concats drugs and targets vectors
    
    Args: 
        df: A DataFrame of drug-target interactions
        embedding: A DataFrame of embedded vectors for each drug and target
        
    Returns: A DataFrame of drug-target interaction vectors
    """
    dataset = []
    for idx, row in df.iterrows():
        from_vector = embedding.loc[row['from']]
        to_vector = embedding.loc[row['to']]
        features = from_vector.append(to_vector).reset_index(drop=True)
        features = features.append(row)
        dataset.append(features)

    df_final = pd.DataFrame(dataset)
    df_final.drop(columns=['from', 'to'], inplace=True)
    return df_final

In [ ]:
new_negative_DTI = negative_DTI[['from','to']]
new_negative_DTI['label'] = [['negative_DTI']] * len(new_negative_DTI)

In [ ]:
df_negative_final = concating(new_negative_DTI, embeddings_all)

In [ ]:
df_final = concating(new_DTI, embeddings_all)

In [ ]:
## create matrix of labels as 0 and 1 (one hot encoder)
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

df_label = pd.DataFrame(mlb.fit_transform(df_final['label']), columns=mlb.classes_, index=df_final.index)

In [ ]:
df_label_new  = df_label.copy()
df_label_new['sum'] = df_label.sum(axis=1) 
df_label_new['sum'].value_counts()
df_label_new ['label'] = df_final['label']

In [ ]:
list_label = []
for i in df_label_new.index:
    if df_label_new.loc[[i]]['sum'][i]== 1:
        list_label.insert(len(list_label),str(df_label_new.loc[[i]]['label'][i][0]))
    else:
        list_label.insert(len(list_label),df_label_new.loc[[i]]['sum'][i])

In [ ]:
df_label_new['final'] = list_label
df_label_new['final'].value_counts()

In [ ]:
## add new label format to gene/drug interaction dataframe
df_interaction_newLabel = pd.concat([df_final.drop(columns=['label']), df_label], axis=1).drop_duplicates()

In [ ]:
for i in  selected_type:
    print(i)
    sns.set(style = "white")
    color_dict = {'decreases^activity':'#5C5D9E',
     'decreases^expression':'#317EC2',
     'decreases^reaction':'#5AAA46',
     'increases^activity':'#C03830',
     'increases^expression':'#E7872B',
     'increases^reaction':'#E9C61D'}

    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection = '3d')
    df_pca = df_interaction_newLabel.copy()
    pca_result = pca.fit_transform(df_interaction_newLabel.drop(columns=selected_type).values)

    x = pca_result[:,0]
    y = pca_result[:,1]
    z = pca_result[:,2]
    df_pca['type']= df_pca[i]
    
    color = color_dict[i]

    cmap_dict = {0: "silver", 1: color}
    cmap = ListedColormap(list(cmap_dict.values()))

    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_zlabel("")

    sc = ax.scatter(x, y, z, s=40, cmap=cmap, c=df_pca['type'], marker='o', alpha=1)

    plt.show()

In [ ]:
for i in  selected_type:   
    df_pca = df_interaction_newLabel.copy()
    pca = PCA(n_components=3)
    pca_result = pca.fit_transform(df_interaction_newLabel.drop(columns=selected_type).values)


    df_pca['PCA-1'] = pca_result[:,0]
    df_pca['PCA-2'] = pca_result[:,1]
    df_pca['PCA-3'] = pca_result[:,2]

    df_pca['type']= df_pca[i]
    chart = ggplot(df_pca, aes(x='PCA-1', y='PCA-3',  color='factor(type)') ) \
        + geom_point(size=120, alpha=0.8) 

    chart.show()

### Removing noise labels

In [ ]:
expression_noise = list(df_interaction_newLabel.loc[(df_interaction_newLabel['decreases^expression']==1) & (df_interaction_newLabel['increases^expression']==1), :].index)
reaction_noise =  list(df_interaction_newLabel.loc[(df_interaction_newLabel['decreases^reaction']==1) & (df_interaction_newLabel['increases^reaction']==1), :].index)
activity_noise =  list(df_interaction_newLabel.loc[(df_interaction_newLabel['decreases^activity']==1) & (df_interaction_newLabel['increases^activity']==1), :].index)

In [ ]:
all_noise = expression_noise+reaction_noise+activity_noise
print(f'number of noise: {len(all_noise)}')
 
df_interaction_newLabel = df_interaction_newLabel.drop(all_noise)
df_interaction_newLabel

### Report

In [ ]:
## some DTI has more than one type of interaction
num_all_interaction = 0
for i in selected_type:
    tmp1 = df_interaction_newLabel[i].sum()
    num_all_interaction = num_all_interaction+tmp1
num_all_interaction

In [ ]:
for i in selected_type:
    n_label = df_interaction_newLabel[i].sum()
    n_rest = len(df_interaction_newLabel)- n_label
    print (f'number of {i}: {n_label} VS. rest: {n_rest}')

In [ ]:
## seperate features (as x) and labels (as y)
def get_data(df_train, label, all_labels):
    df_interaction_new = df_train.copy()
    df_interaction_new['label'] = df_interaction_new[label]
    df_interaction_new.drop(columns=list(all_labels), inplace=True)
    print(df_interaction_new['label'].value_counts())
    
    X = df_interaction_new.drop(columns=['label'])
    y = pd.DataFrame(df_interaction_new['label'])
    
    return X, y.values.ravel()

def get_sample_weight(y_train):
    weight_ratio = float(len(y_train[y_train == 0]))/float(len(y_train[y_train == 1]))
    
    return weight_ratio

In [ ]:
def calculate_metrics(y, y_pred):
    result = {}
    result['accuracy'] = accuracy_score(y, y_pred)

    try:
        result['ROC'] = metrics.roc_auc_score(y, y_pred,average='binary')
    except:
        pass
    
    result['precision'] = precision_score(y, y_pred, average='binary')   
    result['recall'] = recall_score(y, y_pred, average='binary')   
    result['f1_score'] = f1_score(y, y_pred, average='binary')  
    try:
        TN = confusion_matrix(y, y_pred)[0,0]
        FP = confusion_matrix(y, y_pred)[0,1]
        specifity = TN/(TN+FP)
        dic['specifity'] = specifity
    except:
        pass

    result['confusion_matrix'] = confusion_matrix(y, y_pred)
    
    try:
        fpr, tpr, _ = roc_curve(y, y_pred)
        roc_auc = auc(fpr, tpr)
        result['fpr'] = fpr
        result['tpr'] = tpr
        result['roc_auc'] = roc_auc
    except:
        pass
    
    return result

In [ ]:
# split data to train-validation and external test 
DTI_external_test = df_interaction_newLabel.sample(frac = 0.1)
DTI_new = df_interaction_newLabel.drop(DTI_external_test.index)
print (f'DTI: {len(df_interaction_newLabel)}')
print (f'DTI_external_test: {len(DTI_external_test)}')
print (f'DTI_train_internal_test: {len(DTI_new)}')
DTI_external_test.head()

In [ ]:
def summary_results(df, print_results):
    """
    This function create dictinary to reports results 
    
    Args: 
        df: A DataFrame of results for diffrenet runs/CV
    Returns: 
        dic_results: dictionary of all results
    """
    
    dic_results = {}
    
    for label in selected_type:
        
        df_a_labl = df[df['clf_name']==label]
        if print_results:
            print(f'============  {label}  ============')
            print(f"Mean accuracy: {df_a_labl['accuracy'].mean()} ({df_a_labl['accuracy'].std()})")
            print(f"Mean ROC: {df_a_labl['roc_auc'].mean()}({df_a_labl['roc_auc'].std()})")
            print(f"Mean f1_score: {df_a_labl['f1_score'].mean()}({df_a_labl['f1_score'].std()})")
            print(f"Mean precision: {df_a_labl['precision'].mean()}({df_a_labl['precision'].std()})")
            print(f"Mean recall: {df_a_labl['recall'].mean()}({df_a_labl['recall'].std()})\n")
        
        dic_results[label] = {'accuracy':df_a_labl['accuracy'].mean(), 
                             'precision':df_a_labl['precision'].mean(),
                             'recall':df_a_labl['recall'].mean(),
                             'ROC':df_a_labl['roc_auc'].mean(), 'f1_score':df_a_labl['f1_score'].mean()}

    if print_results:
        print(f'\n  ==================  Total  ==================\n\n\n')
        print(f"Mean accuracy: {df['accuracy'].mean()}({df['accuracy'].std()})")
        print(f"Mean ROC: {df['roc_auc'].mean()} ({df['roc_auc'].std()})")
        print(f"Mean f1_score: {df['f1_score'].mean()}({df['f1_score'].std()})")
        print(f"Mean precision: {df['precision'].mean()}({df['precision'].std()})")
        print(f"Mean recall: {df['recall'].mean()}({df['recall'].std()})\n")
    
    dic_results['total'] = {'accuracy':df['accuracy'].mean(), 
                             'precision':df['precision'].mean(),
                             'recall':df['recall'].mean(),
                             'ROC':df['roc_auc'].mean(), 'f1_score':df['f1_score'].mean()}
    return dic_results

In [ ]:
def balance_df(X, y):
    
    print(f"Before balancing, counts of label '1': {sum(y == 1)}")
    print(f"Before balancing, counts of label '0': {sum(y == 0)}\n")

    saved_cols = X.columns
    sm = SMOTE(random_state=42)
        
    X_res, y_res = sm.fit_sample(X.copy(), y.copy().ravel())
    
    print(f"After balancing, counts of label '1': {sum(y_res == 1)}")
    print(f"After balancing, counts of label '0': {sum(y_res == 0)}")

    return pd.DataFrame(X_res, columns=saved_cols), pd.Series(y_res)

In [ ]:
def classification(df, selected_type, run , n=5, SCALE_POS_RATIO="ratio",
                   early_stopping=10, learning_rate=0.3, depth=6, threshold=0.5,
                   G=0, subsample_ratio=1, alpha=0, obj='reg:squarederror', PR = True):
    """
    This function creates the machine learning model 
    
    Args: 
        df: A DataFrame of drug-target interactions (train and internal-test 90%)
        selected_type: list of labels
        n: Number of folds
        SCALE_POS_RATIO: ratio for imbalnace dataset ("ratio": num_0/num_1/"SMOTE": balance trainset/ 'None':real data/int: give a number --for tuning)
        early_stopping: early_stopping
        learning_rate: learning_rate --> [0,1]
        depth: Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit
        G: gamma
        subsample_ratio: Subsample ratio of the training instances
        PR: print results (True for multi-calss)
    Returns: 
        all_train_results: Traning results for n folds
        all_test_results: Testing results for n folds
        dic_results_train: dic train summary result
        dic_results_test: dic test summary result
    """
    
    num_crossVal = 0 
    all_train_results, all_test_results = [], []
    kf = KFold(n_splits=n, random_state=42, shuffle=True)

    DTI_for_ML = df.copy()
    kf.get_n_splits(DTI_for_ML)

    for train_index, test_index in kf.split(DTI_for_ML):
        df_train, df_test = DTI_for_ML.iloc[train_index], DTI_for_ML.iloc[test_index]
        num_crossVal = num_crossVal + 1 
        print(f'\n\n\n  KFold: {num_crossVal}')
        print(f'# all DTI: {DTI_for_ML.shape[0]}')
        print(f'# DTI train-set: {df_train.shape[0]}')
        print(f'# DTI test-set: {df_test.shape[0]}')

        for label in selected_type:
            print('\n\n=============================================================')
            print(f'training for {label}...')
            X, y = get_data(df_train, label, selected_type)
            print(f'X_train.shape: {X.shape}')
            
            print(f'testing for {label}...')
            X_test, y_test = get_data(df_test, label, selected_type)
            print(f'X_test.shape: {X_test.shape}')

            if SCALE_POS_RATIO == 'ratio':
                weight_ratio = get_sample_weight(y)
                model = XGBClassifier(scale_pos_weight=weight_ratio, eta=learning_rate, max_depth= depth,
                                      gamma= G, subsample=subsample_ratio, reg_alpha=alpha, objective=obj)
                
            elif SCALE_POS_RATIO == 'SMOTE':
                model = XGBClassifier(eta=learning_rate, max_depth= depth, gamma= G, subsample=subsample_ratio, 
                                      reg_alpha=alpha, objective=obj)
                X, y = balance_df(X, y)
            
            elif SCALE_POS_RATIO == 'None':
                model = XGBClassifier(eta=learning_rate, max_depth= depth, gamma= G, subsample=subsample_ratio,
                                     reg_alpha=alpha, objective=obj)
                
            else:
                model = XGBClassifier(scale_pos_weight=SCALE_POS_RATIO, eta=learning_rate, max_depth= depth,
                                      gamma= G, subsample=subsample_ratio, reg_alpha=alpha, objective=obj)
            
            model.fit(X, y, early_stopping_rounds=early_stopping, eval_metric=['logloss', 'auc'], eval_set=[(X_test, y_test)], verbose=True)
            #PCA_label (X, y)
            #print(f'Label: {label}\n\n')

            # predic probability
            pred_train_proba = model.predict_proba(X)[:,1]
            # map to binary
            pred_train = list(np.where(pred_train_proba> threshold, 1, 0))
            
            train_results = calculate_metrics(y, pred_train)
            train_results['clf_name'] = label
            train_results['model'] = model
            train_results['Kfold'] = num_crossVal
            train_results['Run'] = run+1

            all_train_results.append(train_results)
           
            
            # predic probability
            pred_test_proba = model.predict_proba(X_test)[:,1]
            # map to binary
            pred_test = list(np.where(pred_test_proba>threshold, 1, 0))
            
            test_results = calculate_metrics(y_test, pred_test)
            test_results['clf_name'] = label
            test_results['model'] = model
            test_results['Kfold'] = num_crossVal
            test_results['Run'] = run+1
            all_test_results.append(test_results)
            print(f'test_results: {test_results}')
            
    df_all_test_results = pd.DataFrame(all_test_results)
    df_all_train_results = pd.DataFrame(all_train_results)  
    
    
        
    print('train summary results')
    dic_results_train = summary_results(df_all_train_results, print_results=PR)

    print('internal-test summary results')
    dic_results_test = summary_results(df_all_test_results, print_results=PR)
    
    return all_train_results, all_test_results, dic_results_train, dic_results_test

In [ ]:
THRESHOLD = 0.5 
RUN = 5
FOLD = 5
LEARNING_RATE = 0.01
BALANCING = 'SMOTE' #['None','SMOTE','ratio',int],
STOPPING = 20
DEPTH = 5
GAMMA = 1
SUBSAMPLE = 1
ALPHA = 0
OBJECTIVE ='reg:squarederror'

In [ ]:
# XGBoost
resuts_all_runs_test = []
resuts_all_runs_train = []
resuts_external_list = []

for run in range(0, RUN):
    # 10% as external test set, 90% internal-test and train
    DTI_external_test = df_interaction_newLabel.sample(frac = 0.1)
    DTI_new = df_interaction_newLabel.drop(DTI_external_test.index)

    all_train_results, all_test_results, dic_results_train, dic_results_test = classification (DTI_new, selected_type, run=RUN, n=FOLD, SCALE_POS_RATIO=BALANCING, 
                                                                                               early_stopping=STOPPING, learning_rate=LEARNING_RATE, depth=DEPTH,
                                                                                               threshold=THRESHOLD, G=GAMMA, subsample_ratio= SUBSAMPLE, alpha=ALPHA,
                                                                                              obj=OBJECTIVE, PR=True )                                                                                                                                                                                                                                                                               

    # find the best model on internal testset and apply on external testset
    for i in selected_type:
        df_test_all = pd.DataFrame(all_test_results)
        best_model_for_a_label = list(df_test_all[df_test_all['clf_name']==i].sort_values(by=['f1_score'], ascending=False)['model'])[0]

        X_ex, y_ex = get_data(DTI_external_test, i, selected_type)
        ## hello
        X_ex, y_ex = balance_df(X_ex, y_ex)
        ## hello
        pred_extest_proba = best_model_for_a_label.predict_proba(X_ex)[:,1]
        pred_extest = list(np.where(pred_extest_proba>THRESHOLD, 1, 0))

        extest_results = calculate_metrics(y_ex, pred_extest)
        extest_results['Run'] = run+1
        extest_results['clf_name'] = i
        extest_results['best_model'] = best_model_for_a_label

        resuts_external_list.append(extest_results)
        
    resuts_all_runs_test = resuts_all_runs_test + all_test_results
    resuts_all_runs_train = resuts_all_runs_train + all_train_results

#### External-test

In [ ]:
df_ex_results = pd.DataFrame(resuts_external_list)
dic_ex_results = summary_results(df_ex_results, print_results=True)

#### Internal-test

In [ ]:
summary_of_test = summary_results(pd.DataFrame(all_test_results), print_results=True)

#### Train

In [ ]:
summary_of_train = summary_results(pd.DataFrame(all_train_results), print_results=True)

In [ ]:
# Report
for i in selected_type:
    n_label = df_interaction_newLabel[i].sum()
    n_rest = len(df_interaction_newLabel)- n_label
    print (f'number of {i}: {n_label} VS. rest: {n_rest}')

### Unkown drug-targets

In [ ]:
def all_unknown_drug(set_all_drug, set_all_target, dic_all_interactions):
    """
    This function create all unkown drug-target interactions and save it (.pkl)
    
    Args: 
        set_all_drug: list of unique drug names
        set_all_target: list of unique target names
        dic_all_interactions: dictionary of all known drug-target interactions    
    """
    
    first = 0
    step = 50000
    len_total = 0 
    # Sinece the data is big --> spilit it to small parts and save them separately
    for i in range(step, len(set_all_drug), step):
        set_all_drug_temp = set_all_drug[first:i]
        all_unknown_interactions = {(x,y) for x in set_all_drug_temp for y in set_all_target}
        all_unknown_interactions.difference_update(dic_all_interactions.keys())
        with open(f'unknown_interactions_'+f'{first}_{i}.pkl', 'wb') as fp:
            pickle.dump(all_unknown_interactions, fp)
        print(f'\nindex: {first}_{i}')
        print(f'len: {len(all_unknown_interactions)}')
        first = i 
        len_total += len(all_unknown_interactions)
        del all_unknown_interactions 
        del set_all_drug_temp

    if first < len(set_all_drug):
        set_all_drug_temp = set_all_drug[first:len(set_all_drug)]
        all_unknown_interactions = {(x,y) for x in set_all_drug_temp for y in set_all_target}
        all_unknown_interactions.difference_update(dic_all_interactions.keys())
        with open(f'unknown_interactions_'+ f'{first}_{len(set_all_drug)}.pkl', 'wb') as fp:
            pickle.dump(all_unknown_interactions, fp)
        print(f'\nindex: {first}_{len(set_all_drug)}')
        print(f'len: {len(all_unknown_interactions)}')
        len_total += len(all_unknown_interactions)
        del all_unknown_interactions 
        del set_all_drug_temp
   
    del dic_all_interactions ## delete**: have a df of dic_all_interactions --> df_all_DTI
    print(f'\nlen unkown DTI: {len_total}')
    return None

In [ ]:
with open('unknown_interactions_0_276.pkl', 'rb') as f:
    unknown_interactions = pickle.load(f)
df_unknown_DTI = pd.DataFrame(unknown_interactions, columns= ['from','to'])
df_unknown_DTI

In [ ]:
# selecting interaction (remove noises)
new_DTI_tmp = new_DTI.reset_index().drop(columns= ['index'])
new_DTI_tmp = pd.concat([new_DTI_tmp,df_label], axis=1)

all_known_int = new_DTI_tmp[new_DTI_tmp.index.isin(df_interaction_newLabel.index)]
set_all_target = list(set(all_known_int['to']))
set_all_drug = list(set(all_known_int['from']))

In [ ]:
all_known_int['tuple_of_int'] = all_known_int[['from', 'to']].apply(tuple, axis=1)
dic_all_interactions = pd.Series(all_known_int['decreases^activity'].values, index=all_known_int['tuple_of_int']).to_dict()

In [ ]:
sample1 = all_known_int[all_known_int['decreases^reaction']==1]
sample1['from']= sample1['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")

### Report

In [ ]:
for i in selected_type:
    n_label = df_interaction_newLabel[i].sum()
    n_rest = len(df_interaction_newLabel)- n_label
    print (f'number of {i}: {n_label} VS. rest: {n_rest}')

In [ ]:
all_unknown_drug(set_all_drug, set_all_target, dic_all_interactions)
print(f'Number of target: {len(set_all_target)}')
print(f'Number of drug: {len(set_all_drug)}')
print(f'Number of DTI: {len(all_known_int)}')

In [ ]:
## All interactions
with open('all_nod2vec_new.pkl', 'rb') as f:
    all_nod2vec_new = pickle.load(f)

with open('diseaseId2numId_nod2vec.pkl', 'rb') as f:
    disId2numId = pickle.load(f)
numId2dis = pd.Series(disId2numId.keys(), index=disId2numId.values()).to_dict()

with open('disid2disname.pkl', 'rb') as f:
    disid2disname = pickle.load(f)

### Diseas without drug 

In [ ]:
all_disease = set(list(all_nod2vec_new[all_nod2vec_new['Type_Interaction']=='DisD']['to'])+list(all_nod2vec_new[all_nod2vec_new['Type_Interaction']=='DisT']['from']))

DisD_interactions = all_nod2vec_new[all_nod2vec_new['Type_Interaction']=='DisD']
dis_with_dug = set(DisD_interactions['to'])
dis_without_drug = all_disease - dis_with_dug

print(f'number of  all disease : {len(all_disease)}')
print(f'number of disease without drug: {len(dis_without_drug)}')
print(f'number of disease with drug: {len(dis_with_dug)}')

In [ ]:
print('name of diseases without known drug: \n')
name_diseses_without_drug = list(pd.DataFrame(dis_without_drug)[0].map(numId2dis).map(disid2disname))

In [ ]:
## target for the diseases without drug
DisT_interactions = all_nod2vec_new[all_nod2vec_new['Type_Interaction']=='DisT']
DisT_for_diseases_without_drug = DisT_interactions[DisT_interactions['from'].isin(list(dis_without_drug))]

In [ ]:
# dis embediing vectors
embeddings_dis = embeddings_all[embeddings_all.index.isin(all_disease)]
embeddings_dis.index = embeddings_dis.index.map(numId2dis) #.map(disid2disname)

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(
    n_clusters=5, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0)

y_km = km.fit_predict(embeddings_dis)

embeddings_dis_km = embeddings_dis.copy()
embeddings_dis_km['km'] = y_km

In [ ]:
# disese name and cluster
dises_cluster = embeddings_dis[[1]]
dises_cluster['cluster'] = y_km
dises_cluster['dis'] = dises_cluster.index
dises_cluster = dises_cluster[['cluster', 'dis']]
dises_cluster.head()

In [ ]:
PCA_label(embeddings_dis, y_km)

In [ ]:
disname2disid = pd.Series(disid2disname.keys(), index =disid2disname.values()).to_dict()
dis2numId = pd.Series(numId2dis.keys(), index =numId2dis.values()).to_dict()

### Developing model for expeimental data


In [ ]:
SCALE_POS_RATIO=BALANCING
early_stopping=STOPPING
learning_rate=LEARNING_RATE
depth=DEPTH
threshold=THRESHOLD
G=GAMMA
subsample_ratio= SUBSAMPLE
alpha=ALPHA
obj=OBJECTIVE

In [ ]:
# First step for new DTI prediction --> predict positive or negative
df_final_tmp = df_final.iloc[:,:-1]
df_final_tmp['type_int'] = 1

# negative interactions
df_negative_final_tmp = df_negative_final.iloc[:, :-1]
df_negative_final_tmp['type_int'] = 0
df_poditive_negative_DTI = df_final_tmp.append(df_negative_final_tmp).reset_index(drop=True)

df_poditive_negative_DTI['type_int'].value_counts()

In [ ]:
df_poditive_negative_DTI1 = df_poditive_negative_DTI.copy()
pca = PCA(n_components=3)
pca_result = pca.fit_transform(df_poditive_negative_DTI.drop(columns= ['type_int']).dropna().values)


df_poditive_negative_DTI1['PCA-1'] = pca_result[:,0]
df_poditive_negative_DTI1['PCA-2'] = pca_result[:,1]
df_poditive_negative_DTI1['PCA-3'] = pca_result[:,2]

In [ ]:
df_poditive_negative_DTI1['type_int'].value_counts()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection = '3d')

x = df_poditive_negative_DTI1['PCA-1']
y = df_poditive_negative_DTI1['PCA-2']
z = df_poditive_negative_DTI1['PCA-3']

cmap_dict = {0: "red", 1: "green"}
cmap = ListedColormap(list(cmap_dict.values()))

ax.set_xlabel(" ")
ax.set_ylabel(" ")
ax.set_zlabel(" ")

sc = ax.scatter(x, y, z, s=30,cmap=cmap, c=df_poditive_negative_DTI['type_int'], marker='o', alpha=1)

plt.show()


In [ ]:
PCA_label(df_poditive_negative_DTI.drop(columns= ['type_int']),df_poditive_negative_DTI['type_int'] )

In [ ]:
def DT2Vec(df_poditive_negative_DTI, SCALE_POS_RATIO=BALANCING, early_stopping=STOPPING,
           learning_rate=LEARNING_RATE, depth=DEPTH, threshold=THRESHOLD, G=GAMMA, subsample_ratio= SUBSAMPLE,
           alpha=ALPHA, obj=OBJECTIVE, mode_test=False):
    
    """
    This function is DT2Vec method, used for predincting positive and negative interactions
    
    Args: 
        df_poditive_negative_DTI: df of embedded vectors of postive and negative interactions
        SCALE_POS_RATIO: ratio for imbalnace dataset ("ratio": num_0/num_1/"SMOTE": balance trainset/ 'None':real data/int: give a number --for tuning)
        early_stopping: early_stopping
        learning_rate: learning_rate --> [0,1]
        depth: Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit
        G: gamma
        subsample_ratio: Subsample ratio of the training instances
        mode_test: True for evalutaing method (test, train, validation), False for developing method (train and test)  
    """
    
    # XGBoost
    resuts_all_runs_test = []
    resuts_all_runs_train = []
    resuts_external_list = []
    selected_type = ['type_int']
    
    for run in range(0, RUN):
        if mode_test:
            # 10% as external test set, 90% internal-test and train
            DTI_external_test = df_poditive_negative_DTI.sample(frac = 0.1)
            DTI_new = df_poditive_negative_DTI.drop(DTI_external_test.index)
        else:
            DTI_new = df_poditive_negative_DTI.copy()

        all_train_results, all_test_results, dic_results_train, dic_results_test = classification (DTI_new, selected_type, run=RUN, n=FOLD, SCALE_POS_RATIO=BALANCING, 
                                                                                                   early_stopping=STOPPING, learning_rate=LEARNING_RATE, depth=DEPTH,
                                                                                                   threshold=THRESHOLD, G=GAMMA, subsample_ratio= SUBSAMPLE, alpha=ALPHA,
                                                                                                  obj=OBJECTIVE, PR=False)                                                                                                                                                                                                                                                                               

        # find the best model on internal testset and apply on external testset
        if mode_test:
            for i in selected_type:
                df_test_all = pd.DataFrame(all_test_results)
                best_model_for_a_label = list(df_test_all[df_test_all['clf_name']==i].sort_values(by=['f1_score'], ascending=False)['model'])[0]

                X_ex, y_ex = get_data(DTI_external_test, i, selected_type)
                 ## hello
                X_ex, y_ex = balance_df(X_ex, y_ex)
                ## hello
                pred_extest_proba = best_model_for_a_label.predict_proba(X_ex)[:,1]
                pred_extest = list(np.where(pred_extest_proba>THRESHOLD, 1, 0))

                extest_results = calculate_metrics(y_ex, pred_extest)
                extest_results['Run'] = run+1
                extest_results['clf_name'] = i
                extest_results['best_model'] = best_model_for_a_label

                resuts_external_list.append(extest_results)

        resuts_all_runs_test = resuts_all_runs_test + all_test_results
        resuts_all_runs_train = resuts_all_runs_train + all_train_results
    
    return pd.DataFrame(resuts_all_runs_test), pd.DataFrame(resuts_all_runs_train), pd.DataFrame(resuts_external_list)

In [ ]:
df_poditive_negative_DTI

In [ ]:
# testing the method by having test, train and external test
#df_poditive_negative_DTI = df_poditive_negative_DTI.drop(columns= ['PCA-1','PCA-2','PCA-3'])
resuts_all_runs_tPN_test, resuts_all_runs_train_tPN, resuts_external_list_tPN = DT2Vec(df_poditive_negative_DTI, mode_test=True)

In [ ]:
resuts_external_list_tPN

In [ ]:
print(resuts_external_list_tPN['accuracy'].mean())
print(resuts_external_list_tPN['accuracy'].std())
print(resuts_external_list_tPN['f1_score'].mean())
print(resuts_external_list_tPN['f1_score'].std())
print(resuts_external_list_tPN['precision'].mean())
print(resuts_external_list_tPN['precision'].std())

In [ ]:
# developing model for predicting positive and negative interactions
dic_model = {}

resuts_all_runs_test_PN, resuts_all_runs_train_PN, resuts_external_list_PN = DT2Vec(df_poditive_negative_DTI)

# selecte best model with highest f1_score
best_PN_model = resuts_all_runs_test_PN.sort_values(by=['f1_score'], ascending=False)['model'][0]
dic_model['postive/negative'] = best_PN_model

In [ ]:
# XGBoost 
resuts_all_runs_test_final = []
resuts_all_runs_train_final = []

for run in range(0, RUN):
    # 10% as external test set, 90% internal-test and train
    DTI_new_final = df_interaction_newLabel.copy()

    all_train_results_final, all_test_results_final, dic_results_train_final, dic_results_test_final = classification (DTI_new_final, selected_type, run=RUN, n=FOLD, SCALE_POS_RATIO=BALANCING, 
                                                                                               early_stopping=STOPPING, learning_rate=LEARNING_RATE, depth=DEPTH,
                                                                                               threshold=THRESHOLD, G=GAMMA, subsample_ratio= SUBSAMPLE, alpha=ALPHA,
                                                                                              obj=OBJECTIVE)                                                                                                                                                                                                                                                                               
    resuts_all_runs_test_final = resuts_all_runs_test_final + all_train_results_final
    resuts_all_runs_train_final = resuts_all_runs_train_final + all_test_results_final
    
# find the best model 
for i in selected_type:
    df_test_all_final = pd.DataFrame(resuts_all_runs_test_final)
    best_model_for_a_label_final = list(df_test_all_final[df_test_all_final['clf_name']==i].sort_values(by=['f1_score'], ascending=False)['model'])[0]

    dic_model[i] = best_model_for_a_label_final

# Predicting on unkown DTIs

In [ ]:
# Unkown drug-target
df_unknown_DTI.head()

In [ ]:
# BIRC5
df_unknown_DTI[df_unknown_DTI['to']==332]

In [ ]:
with open('vector_unknown_DTI.pkl', 'rb') as f:
    vector_unknown_DTI = pickle.load(f)
vector_unknown_DTI

In [ ]:
# prdict P/N interactation labels for unkown DTIs
pred_proba_PosNeg = dic_model['postive/negative'].predict_proba(vector_unknown_DTI)[:,1]

In [ ]:
# adding Pos/Neg labels and drug/target names
vector_unknown_DTI_label = vector_unknown_DTI.copy()
vector_unknown_DTI_label['Pos/Neg'] = pred_proba_PosNeg
vector_unknown_DTI_label['from'] = df_unknown_DTI['from']
vector_unknown_DTI_label['to'] = df_unknown_DTI['to']
vector_unknown_DTI_label = vector_unknown_DTI_label.sort_values(by=['Pos/Neg'], ascending=False)
vector_unknown_DTI_label                                            

In [ ]:
# Select highy positive interactions 
highy_pos_DTI = vector_unknown_DTI_label[vector_unknown_DTI_label['Pos/Neg']>=0.95]
highy_pos_DTI

In [ ]:
highy_pos_DTI[highy_pos_DTI['to']== 7157]

In [ ]:
unkown_DTI_labels = highy_pos_DTI.copy()
for i in selected_type:
    new_label = dic_model[i].predict_proba(highy_pos_DTI.drop(columns=['Pos/Neg','from','to']))[:,1]
    unkown_DTI_labels[i] = new_label
unkown_DTI_labels

In [ ]:
# selecting 110 dtis in each labels
dic_new_DTIs = {}
unkown_DTI_labels['from_to_tuples'] = list(unkown_DTI_labels[['from', 'to']].to_records(index=False))

for i in selected_type:
    dic_new_DTIs[i] = unkown_DTI_labels.sort_values(by=[i], ascending=False)[['from','to','Pos/Neg','from_to_tuples',i]][:110]
dic_new_DTIs

In [ ]:
# Finding Wege interactions : increases^expression and'decreases^expression' or 'decreases^reaction', 'increases^reaction' or 'increases^activity', 'decreases^activity'
expression_dub = [name for name in list(dic_new_DTIs['increases^expression']['from_to_tuples']) if name in list(dic_new_DTIs['decreases^expression']['from_to_tuples']) ]
reaction_dub = [name for name in list(dic_new_DTIs['increases^reaction']['from_to_tuples']) if name in list(dic_new_DTIs['decreases^reaction']['from_to_tuples']) ]
activity_dub = [name for name in list(dic_new_DTIs['increases^activity']['from_to_tuples']) if name in list(dic_new_DTIs['decreases^activity']['from_to_tuples']) ]
activity_dub

In [ ]:
dic_new_DTIs['increases^activity'] = dic_new_DTIs['increases^activity'][~dic_new_DTIs['increases^activity']['from_to_tuples'].isin(activity_dub)][:100]
dic_new_DTIs['decreases^activity'] = dic_new_DTIs['decreases^activity'][~dic_new_DTIs['decreases^activity']['from_to_tuples'].isin(activity_dub)][:100]

dic_new_DTIs['increases^activity'] = dic_new_DTIs['increases^activity'][:100]
dic_new_DTIs['decreases^activity'] = dic_new_DTIs['decreases^activity'][:100]

In [ ]:
dic_new_DTIs_new  = dic_new_DTIs.copy()
for i in selected_type:
    dic_new_DTIs[i]['ChEMBL id'] = dic_new_DTIs[i]['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
    dic_new_DTIs[i]['Entrez Gene id']= dic_new_DTIs[i]['to'].map(entrezgene_dic)
    j = i.replace('^', '_')
    dic_new_DTIs[i].to_csv(f'DTI2VecPLUS/{j}.csv')
dic_new_DTIs

In [ ]:
new_decreases_expression = dic_new_DTIs['decreases^expression']
new_decreases_expression['drug_id'] = new_decreases_expression['ChEMBL id'].map(chmble2drugname)
new_decreases_expression

In [ ]:
bibpart_plot = pd.DataFrame({'from':[0],'to':[0], 'Pos/Neg':[0], 'from_to_tuples':[0],
              'ChEMBL id':[0], 'Entrez Gene id':[0], 'drug_id':[0], 'type_interaction':[0]})
for i in selected_type:
    tmp = dic_new_DTIs[i][['from','to','Pos/Neg','from_to_tuples','ChEMBL id','Entrez Gene id']][:20]
    tmp['drug_id'] = tmp['ChEMBL id'].map(chmble2drugname)
    tmp['type_interaction'] = i
    bibpart_plot = bibpart_plot.append(tmp)
bibpart_plot = bibpart_plot[1:]
bibpart_plot['bi-1']= 1
bibpart_plot['bi-0']= 0
bibpart_plot = bibpart_plot.sort_values(by= 'drug_id')
bibpart_plot['main']=bibpart_plot['drug_id']+bibpart_plot['Entrez Gene id'] 
bibpart_plot

In [ ]:
len(set(bibpart_plot['drug_id']))

In [ ]:
bibpart_plot[['drug_id','Entrez Gene id','bi-1','bi-0']].values.tolist()

In [ ]:
color_dict = {'decreases^activity':'#5C5D9E',
 'decreases^expression':'#317EC2',
 'decreases^reaction':'#5AAA46',
 'increases^activity':'#C03830',
 'increases^expression':'#E7872B',
 'increases^reaction':'#E9C61D'}

In [ ]:
bibpart_plot['color']= bibpart_plot['type_interaction'].map(color_dict)
color_dict_final = pd.Series(bibpart_plot['color'].values, index=bibpart_plot['main'] ).to_dict()
color_dict_final

In [ ]:
bibpart_plot[['drug_id','Entrez Gene id', 'color']].to_csv('chart.csv')

## Disease without drug

In [ ]:
def drug_reperpusing_dis(embeddings_all, df_unknown_DTI, numId2drugId, drug_cut2chem, dic_model,
                         target_gene, pre_list, TH):
    """
    This function use for drug repupusing
    
    Args: 
        embeddings_all: Data frame of embedding vectors
        df_unknown_DTI: Data frame of unkown DTI
        numId2drugId: dict of id to drug ids
        drug_cut2chem: dict of drug ids
        dic_model: dictinary of models
        target_gene: a target gene
        pre_list: type of interactions
        TH: Thereashod of probability
        
    Return:  Dictianary of new dug and it probability
        
        """
    dic_new_dug = {}
    selected_gene = df_unknown_DTI[df_unknown_DTI['to']==target_gene]
    df_vector_interactions = concating(selected_gene, embeddings_all)

    # finding positive interactions using DT2Vec
    pred_proba_PN = dic_model['postive/negative'].predict_proba(df_vector_interactions)[:,1]
    pred_PN = list(np.where(pred_proba_PN>=TH, 1, 0))

    df_all = pd.concat([df_vector_interactions, selected_gene.reset_index(drop=True)], axis=1)
    df_all['drug'] = df_all['from'].map(numId2drugId).map(drug_cut2chem)

    df_all['label_proba_PN'] = pred_proba_PN
    df_all['label_PN'] = pred_PN

    for i in pre_list:
        pred_proba_a_label = dic_model[i].predict_proba(df_vector_interactions)[:,1]
        df_all[f'{i}_proba'] = pred_proba_a_label    
        pred_a_label = list(np.where(pred_proba_a_label>=TH, 1, 0))
        df_all[i] = pred_a_label


        # tuple of drug and predicted probability 
        df_all[f'drug_proba_{i}'] = df_all[['drug', f'{i}_proba']].apply(tuple, axis=1)

        # positive intearctions
        df_all_p = df_all[df_all['label_PN']==1]

        dic_new_dug[i] = list(df_all_p[df_all_p[i]==1][f'drug_proba_{i}'])
    return dic_new_dug

### COVID-19 

In [ ]:
# list of gene for covid not available in the graph
AllDisGene[(AllDisGene['MESH:D000014']=='MESH:D000086382') & (AllDisGene['Unnamed: 4']=='marker/mechanism')]

In [ ]:
## Covid clusters
cluster_4 = embeddings_dis_km[embeddings_dis_km['km']==4].sort_values(by=[0])
cluster_4['dis']= cluster_4.index.map(disid2disname)
list(cluster_4['dis'])

In [ ]:
COVID_19 = DisT_for_diseases_without_drug[DisT_for_diseases_without_drug['from']==100506959]
COVID_19

In [ ]:
# drugs that can bind to covid's genes
covid_genes = COVID_19['to']
DTI_covid_genes = all_known_int[all_known_int['to'].isin(covid_genes)]
DTI_covid_genes['from'] = DTI_covid_genes['from'].map(numId2drugId).map(drug_cut2chem)
DTI_covid_genes

In [ ]:
drugId2numId_nod2vec[41286]

In [ ]:
# drug id name based on Chmble id 
numId2drugId

### ACE2
- Finding new drugs that can bind to this genes

In [ ]:
DTI[DTI['to']==59272]

In [ ]:
target_gene = 59272

In [ ]:
dic_59272 = {}
df_all_59272 = pd.DataFrame(data={'from': [0], 'to': [0], 'type':[0], 'pro':[0]})
repurpusing_59272_new = unkown_DTI_labels[unkown_DTI_labels['to']==target_gene] 
repurpusing_59272_new['from'] = repurpusing_59272_new['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
for i in ['increases^activity', 'increases^expression', 'increases^reaction']:
    dic_59272[i] = repurpusing_59272_new[repurpusing_59272_new [i]>=0.5].sort_values(by=[i], ascending=False)
    print(dic_59272[i][['Pos/Neg','from','to',i]].head(5))
    print('')
    df_all_59272_tmp = dic_59272[i][['from','to']]
    df_all_59272_tmp['type'] = i
    df_all_59272_tmp['pro'] = dic_59272[i][i]
    df_all_59272 = df_all_59272.append(df_all_59272_tmp)
df_all_59272 = df_all_59272[1:].sort_values(by=['pro'], ascending=False)

In [ ]:
# top 50 drug for
df_all_59272.head(50)

### BSG

In [ ]:
target_gene = 682
pre_list = ['decreases^activity', 'decreases^expression', 'decreases^reaction']

In [ ]:
dic_682 = {}
df_all_682 = pd.DataFrame(data={'from': [0], 'to': [0], 'type':[0], 'pro':[0]})
repurpusing_682_new = unkown_DTI_labels[unkown_DTI_labels['to']==target_gene] 
repurpusing_682_new['from'] = repurpusing_682_new['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
for i in pre_list:
    dic_682[i] = repurpusing_682_new[repurpusing_682_new [i]>=0.5].sort_values(by=[i], ascending=False)
    print(dic_682[i][['Pos/Neg','from','to',i]].head(5))
    print('')
    df_all_682_tmp = dic_682[i][['from','to']]
    df_all_682_tmp['type'] = i
    df_all_682_tmp['pro'] = dic_682[i][i]
    df_all_682 = df_all_682.append(df_all_682_tmp)
df_all_682 = df_all_682[1:].sort_values(by=['pro'], ascending=False)

In [ ]:
df_all_682.head(50)

## DDS 

In [ ]:
def dds_newdrug (kownDrug=[100506944,100506799], num=100):
    newDrugList = list(df_all_682.head(num)['from'].str.replace('CHEMBL', '').astype(int).map(drugId2numId_nod2vec))
    unknow_known_DDS = allDDS[(allDDS['from'].isin(kownDrug)|allDDS['to'].isin(kownDrug)) & (allDDS['to'].isin(newDrugList)|allDDS['from'].isin(newDrugList))].sort_values(by=['weight'], ascending=False)
    unknow_known_DDS['from_id'] = unknow_known_DDS['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
    unknow_known_DDS['to_id'] = unknow_known_DDS['to'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
    unknow_known_DDS['from_name'] = unknow_known_DDS['from_id'].map(chmble2drugname)
    unknow_known_DDS['to_name'] = unknow_known_DDS['to_id'].map(chmble2drugname)


    return unknow_known_DDS

In [ ]:
allDDS = all_nod2vec[all_nod2vec['Type_Interaction']=='DDS']
allDDS

### list of important genes
- EGFR (1956),  BCL2 (596), STAT3 (6774),  MYC (4609), BIRC5 (332)


In [ ]:
gene_id = 1956

In [ ]:
# known interactins
DTI_new_a_gene = DTI[DTI['to']==gene_id]
DTI_new_a_gene['from_id'] = DTI_new_a_gene['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
DTI_new_a_gene['drug_id'] = DTI_new_a_gene['from_id'].map(chmble2drugname)
DTI_new_a_gene

In [ ]:
# Drug can bind
print(list(DTI[DTI['to']==gene_id]['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}").map(chmble2drugname)))
# Disease related 
list(all_nod2vec_new[(all_nod2vec_new['Type_Interaction']=='DisT')& (all_nod2vec_new['to']==gene_id)]['from'].map(numId2dis).map(disid2disname))

In [ ]:
dic_682['decreases^expression']['from'].map(chmble2drugname)

In [ ]:
target_gene=gene_id
pre_list = ['decreases^activity', 'decreases^expression', 'decreases^reaction']
dic_682 = {}
df_all_682 = pd.DataFrame(data={'from': [0], 'to': [0], 'type':[0], 'pro':[0]})
repurpusing_682_new = unkown_DTI_labels[unkown_DTI_labels['to']==target_gene] 
repurpusing_682_new['from'] = repurpusing_682_new['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
for i in pre_list:
    dic_682[i] = repurpusing_682_new[repurpusing_682_new [i]>=0.5].sort_values(by=[i], ascending=False)
    print(dic_682[i][['Pos/Neg','from','to',i]].head(5))
    print('')
    df_all_682_tmp = dic_682[i][['from','to']]
    df_all_682_tmp['type'] = i
    df_all_682_tmp['pro'] = dic_682[i][i]
    df_all_682 = df_all_682.append(df_all_682_tmp)
df_all_682 = df_all_682[1:].sort_values(by=['pro'], ascending=False)
df_all_682['drug_name'] = df_all_682.head(50)['from'].map(chmble2drugname)
df_all_682.head(30)

In [ ]:
STOP HERE

In [ ]:
unknow_known_DDS = dds_newdrug(kownDrug=[100506842,100506802, 100506835, 100506881,100506892, 100506727,100506695], num=100)
unknow_known_DDS

In [ ]:
## drug with highest similsrity
df_all_682[df_all_682['from'].isin(['CHEMBL205596'])]

In [ ]:
STOP HERE

In [ ]:
## drug with highest probability
kownDrug = [100506842,100506802, 100506835, 100506881,100506892, 100506727,100506695]
newDrugList = [drugId2numId_nod2vec[957]]

allDDS[(allDDS['from'].isin(kownDrug)|allDDS['to'].isin(kownDrug)) & (allDDS['to'].isin(newDrugList)|allDDS['from'].isin(newDrugList))].sort_values(by=['weight'], ascending=False)


### BIRC5

In [ ]:
# known interactins
DTI_new_a_gene = DTI[DTI['to']==332]
DTI_new_a_gene['from_id'] = DTI_new_a_gene['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
DTI_new_a_gene

In [ ]:
# Drug can bind to BIRC5
print(list(DTI[DTI['to']==332]['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}").map(chmble2drugname)))
# Disease related to BIRC5
list(all_nod2vec_new[(all_nod2vec_new['Type_Interaction']=='DisT')& (all_nod2vec_new['to']==332)]['from'].map(numId2dis).map(disid2disname))

In [ ]:
target_gene=332
pre_list = ['decreases^activity', 'decreases^expression', 'decreases^reaction']
dic_682 = {}
df_all_682 = pd.DataFrame(data={'from': [0], 'to': [0], 'type':[0], 'pro':[0]})
repurpusing_682_new = unkown_DTI_labels[unkown_DTI_labels['to']==target_gene] 
repurpusing_682_new['from'] = repurpusing_682_new['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
for i in pre_list:
    dic_682[i] = repurpusing_682_new[repurpusing_682_new [i]>=0.5].sort_values(by=[i], ascending=False)
    print(dic_682[i][['Pos/Neg','from','to',i]].head(5))
    print('')
    df_all_682_tmp = dic_682[i][['from','to']]
    df_all_682_tmp['type'] = i
    df_all_682_tmp['pro'] = dic_682[i][i]
    df_all_682 = df_all_682.append(df_all_682_tmp)
df_all_682 = df_all_682[1:].sort_values(by=['pro'], ascending=False)
df_all_682['drug_name'] = df_all_682.head(50)['from'].map(chmble2drugname)
df_all_682.head(30)

In [ ]:
unknow_known_DDS = dds_newdrug(kownDrug=[100506842,100506802, 100506835, 100506881,100506892, 100506727,100506695], num=100)
unknow_known_DDS

In [ ]:
## drug with highest similsrity
df_all_682[df_all_682['from'].isin(['CHEMBL205596'])]

In [ ]:
chmble2drugname['CHEMBL1501']

In [ ]:
## drug with highest probability
kownDrug = [100506842,100506802, 100506835, 100506881,100506892, 100506727,100506695]
newDrugList = [drugId2numId_nod2vec[957]]

allDDS[(allDDS['from'].isin(kownDrug)|allDDS['to'].isin(kownDrug)) & (allDDS['to'].isin(newDrugList)|allDDS['from'].isin(newDrugList))].sort_values(by=['weight'], ascending=False)


### Rare disease

#### Acquired angioedema


In [ ]:
MESH_Acquired_angioedema = disname2disid['Acquired angioedema']
num_Acquired_angioedema = dis2numId[MESH_Acquired_angioedema]
print(f'MESH: {MESH_Acquired_angioedema}, numId:{num_Acquired_angioedema}')

In [ ]:
Acquired_angioedema = DisT_for_diseases_without_drug[DisT_for_diseases_without_drug['from']==100506951]
Acquired_angioedema

In [ ]:
target_gene = 710
pre_list = ['increases^activity', 'increases^expression', 'increases^reaction',
            'decreases^activity', 'decreases^expression', 'decreases^reaction' ]
TH = 0.50

repurpusing_710 = drug_reperpusing_dis(embeddings_all, df_unknown_DTI, numId2drugId, drug_cut2chem, dic_model,
                         target_gene, pre_list, TH)
repurpusing_710

## Heatmap

In [ ]:
# ChEMBL_id to Entrez_id (from uniprot)
with open('ChEMBL2Entrez.pkl', 'rb') as f:
    ChEMBL2Entrez = pickle.load(f)
Entrez2ChEMBL = {y:x for x,y in ChEMBL2Entrez.items()}
Entrez2ChEMBL

In [ ]:
with open('protein2num.pkl', 'rb') as f:
    protein2num = pickle.load(f)
    num2pro = {y:x for x,y in protein2num.items()}

with open('protein_dict.pkl', 'rb') as f:
    protein_dict = pickle.load(f)
    
df_protein_type = pd.DataFrame(protein_dict.items(), columns=['protein', 'type'])
df_protein_type['protein'] = df_protein_type['protein'].map(num2pro)
dic_protein_typ = pd.Series(df_protein_type['type'].values, index=df_protein_type['protein']).to_dict()
dic_protein_typ

In [ ]:
len_list = []
heatmap_tmp = new_DTI.reset_index().drop(all_noise).drop(columns=['index'])
for i in heatmap_tmp.index:
    len_list.append(len(heatmap_tmp['label'][i]))
heatmap_tmp['num_label']= len_list
print('Number of type of interactions:')
heatmap_tmp['num_label'].value_counts()

### Known interactions

In [ ]:
heatmap_pos = heatmap_tmp[['from', 'to', 'num_label']]
heatmap_pos['Chembl']= heatmap_pos['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
heatmap_pos = heatmap_pos.drop(columns=['from'])
heatmap_pos

### known negative interactions

In [ ]:
# (100--> negative)
heatmap_neg = negative_DTI[['Chembl','to']]
heatmap_neg['num_label']= 100
heatmap_neg

### unkown postive prediction

In [ ]:
heatmap_unkown = unkown_DTI_labels[['from','to']+selected_type]
for i in selected_type:
    heatmap_unkown.loc[heatmap_unkown[i] > 0.6, i] = 'pos'
    heatmap_unkown.loc[heatmap_unkown[i] != 'pos', i] = 'neg'
    heatmap_unkown.loc[heatmap_unkown[i] == 'pos', i] = -1
    heatmap_unkown.loc[heatmap_unkown[i] == 'neg', i] = 0
heatmap_unkown

In [ ]:
# 0 --> not in our label --> postive but diffrent postive type like binding
heatmap_unkown['num_label']= heatmap_unkown[selected_type].sum(axis=1)
heatmap_unkown['num_label'].value_counts()

In [ ]:
heatmap_unkown.loc[heatmap_unkown['num_label'] <= -4, 'num_label'] = 0
heatmap_unkown['num_label'].value_counts()

In [ ]:
## both --> increase and deacrse--> change the waGE DTIs to unavailbe or 0
wag_DTIs = heatmap_unkown.copy()
wag_DTIs['expresstion'] = wag_DTIs[['increases^expression','decreases^expression']].sum(axis=1)
wag_DTIs['reaction'] = wag_DTIs[['increases^reaction','decreases^reaction']].sum(axis=1)
wag_DTIs['activity'] = wag_DTIs[['increases^activity','decreases^activity']].sum(axis=1)

wag_DTIs.loc[wag_DTIs['reaction'] == -2, 'num_label'] = 0
wag_DTIs.loc[wag_DTIs['activity'] == -2, 'num_label'] = 0
wag_DTIs.loc[wag_DTIs['expresstion'] == -2, 'num_label'] = 0

In [ ]:
heatmap_perdict = wag_DTIs [['from','to', 'num_label']]
heatmap_perdict['Chembl'] = heatmap_perdict['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")
heatmap_perdict

### unkown Negative interactions

In [ ]:
negative_notHP = vector_unknown_DTI_label[~vector_unknown_DTI_label.index.isin(heatmap_perdict.index)][['from', 'to']]
negative_notHP['num_label']= -100
negative_notHP['Chembl'] = negative_notHP['from'].map(numId2drugId).apply(lambda x: f"CHEMBL{x}")

### All

In [ ]:
list_protein = ['unclassified_protein', 'Trascription_factor', 'Transporter', 'Secreted_protein',
                   'other_categories', 'membrance_receptor', 'Ion_Channel', 'Epigenetic_regulation', 
                    'Enzyme_Cytochrome P450', 'Cytosolic_protein', 'Enzyme_Hydrolase', 'Enzyme_Kinase',
                    'Enzyme_Ligase','Enzyme_Lyase','Enzyme_Oxidoreductase','Enzyme_Phosphatase',
                    'Enzyme_Protease', 'Enzyme_rest', 'Enzyme_Transferase']

protein_type = {}
for p in list_protein:
    df_p = pd.read_csv(f'{p}.csv')
    df_p['type_protein'] = p
dict_protein = pd.Series(df_p['type_protein'].values, index = df_p['ChEMBL ID']).to_dict()
dict_protein['CHEMBL2055'] = 'Trascription_factor'
dict_protein

In [ ]:
# All interactions 
final_heatmap = (negative_notHP.append(heatmap_perdict)).append(heatmap_pos.append(heatmap_neg))[['Chembl','to','num_label']]

In [ ]:
final_heatmap = final_heatmap.rename(columns={"Chembl": "from_ChEMBL", "to": "to_ChEMBL", 'num_label':'label'})
final_heatmap.loc[final_heatmap['label'] == 100, 'label'] = 4
final_heatmap.loc[final_heatmap['label'] == -100, 'label'] = -4

In [ ]:
type_int='label'
list_DTI = final_heatmap[['from_ChEMBL','to_ChEMBL',type_int]].values.tolist()

In [ ]:
result_dict = {}
cols = list(set([x[1] for x in list_DTI]))
for elem in list_DTI:
    result_dict[elem[0]] = [0]*len(cols)

for elem in list_DTI:
    col_idx = cols.index(elem[1])    
    result_dict[elem[0]][col_idx] = elem[2]

df_heatmap = pd.DataFrame.from_dict(result_dict, orient='index', columns=cols)
df_heatmap

In [ ]:
color_list_target = ['#58ACFA','#FF1493', 'yellow','orange', '#00CED1','#5F9EA0','#006400','#96bf65','#fcc808','#7b2b48',
 '#e96957','#e06000','#173679','#d2dd49','#684a6b','#096eb2','#ce482a', 'red', 'lime', 'lightslategray',
                                      'olive', 'rosybrown', 'sienna', 'darkmagenta','midnightblue','maroon',
                                      'lightcoral','gold','sandybrown','tomato','lawngreen','lightgreen','darkorchid',
                                      'lightskyblue','darkgreen']
color_list_drug = ['mediumvioletred','mediumblue','gold','green','violet','mediumturquoise','mediumvioletred','darkgoldenrod','pink','dimgray']
df_hetmap_tmp = df_heatmap.copy()
df_hetmap_tmp = df_hetmap_tmp.T
df_hetmap_tmp['target_color']= df_hetmap_tmp.index.map(Entrez2ChEMBL).map(dic_protein_typ)
df_hetmap_tmp

In [ ]:
final_heatmap['label'].value_counts()
# -4 negative prediction
# 4 known negative
# 0 positive but not in our labels

In [ ]:
sns.set(font_scale=0.7)
# discrete colormap (n samples from a given cmap)
cmap = ['#F5EEF8','#1F618D', '#2E86C1','#85C1E9', 'lightgray','#F5B7B1','#EC7063','#CB4335','#5B2C6F']
plot = sns.clustermap(df_heatmap, row_cluster=True, col_cluster=True, cmap=cmap, figsize=(30, 30)) 
ax = plot.ax_heatmap

In [ ]:
def clustermap(final_DTI, type_int, order= False):   
    list_DTI = final_DTI[['from_ChEMBL','to_ChEMBL',type_int]].values.tolist()

    result_dict = {}
    cols = list(set([x[1] for x in list_DTI]))
    for elem in list_DTI:
        result_dict[elem[0]] = [0]*len(cols)

    for elem in list_DTI:
        col_idx = cols.index(elem[1])    
        result_dict[elem[0]][col_idx] = elem[2]

    df_heatmap = pd.DataFrame.from_dict(result_dict, orient='index', columns=cols)
    
    color_list_target = ['#58ACFA','#FF1493', 'yellow','orange', '#00CED1','#5F9EA0','#006400','#96bf65','#fcc808','#7b2b48',
 '#e96957','#e06000','#173679','#d2dd49','#684a6b','#096eb2','#ce482a', 'red', 'lime', 'lightslategray',
                                      'olive', 'rosybrown', 'sienna', 'darkmagenta','midnightblue','maroon',
                                      'lightcoral','gold','sandybrown','tomato','lawngreen','lightgreen','darkorchid',
                                      'lightskyblue','darkgreen']
    color_list_drug = ['mediumvioletred','mediumblue','gold','green','violet','mediumturquoise','mediumvioletred','darkgoldenrod','pink','dimgray']
    df_hetmap_tmp = df_heatmap.copy()
    df_hetmap_tmp = df_hetmap_tmp.T
    #df_hetmap_tmp['target_color']= df_hetmap_tmp.index.map(Lou_target_new)
    df_hetmap_tmp['target_color']= df_hetmap_tmp.index.map(type_target_new)
    df_hetmap_tmp = df_hetmap_tmp.sort_values(by='target_color').drop(columns='target_color').T
    df_hetmap_tmp['index_id2']= df_heatmap.index.map(Lou_drug_new)
    df_hetmap_tmp = df_hetmap_tmp.sort_values(by='index_id2').drop(columns='index_id2')
    df_heatmap = df_hetmap_tmp.copy()

    df_hetmap_tmp['index_id']= df_hetmap_tmp.index
    my_palette_drug= dict(zip(df_hetmap_tmp.index_id.map(Lou_drug_new).astype(str).unique(),color_list_drug))
    row_colors_drug = df_hetmap_tmp.index_id.map(Lou_drug_new).astype(str).map(my_palette_drug)

    color_dict = {}
    #my_palette_target= dict(zip(set(list(df_hetmap_tmp.rename(columns=Lou_target_new))),color_list_target))
    my_palette_target= dict(zip(set(list(df_hetmap_tmp.rename(columns=type_target_new))),color_list_target))


    df_hetmap_tmp = df_hetmap_tmp.drop(columns=['index_id'])
    for x in list(df_hetmap_tmp):
        #color_dict[x] = my_palette_target[Lou_target_new[x]]
        color_dict[x] = my_palette_target[type_target_new[x]]

    df_hetmap_tmp = df_hetmap_tmp.append(pd.Series(color_dict),ignore_index=True)
    col_colors_target = df_hetmap_tmp[548:].squeeze()

    value_to_int = {'Positive_P': 0, 'Positive': 1, 'Weak': 2, 'Negative_P': 3, 'Negative': 4}
    n = len(value_to_int)     
    sns.set(font_scale=0.7)
    # discrete colormap (n samples from a given cmap)
    cmap = [(173/255, 216/255, 230/255), (0, 0, 139/255), (224/255, 255/255, 255/255), (255/255, 192/255, 203/255), (220/255, 20/255, 60/255)]
    plot = sns.clustermap(df_heatmap.replace(value_to_int),row_cluster=order, col_cluster=order, cmap=cmap, row_colors=row_colors_drug, col_colors=col_colors_target, figsize=(20, 20)) 

    ax = plot.ax_heatmap

    plot.ax_row_dendrogram.set_visible(False)
    plot.ax_col_dendrogram.set_visible(False)
    plot.cax.set_visible(False)
    # modify colorbar:
    colorbar = ax.collections[0].colorbar 
    r = colorbar.vmax - colorbar.vmin 
    colorbar.set_ticks([colorbar.vmin + r / n * (0.5 + i) for i in range(n)])
    colorbar.set_ticklabels(list(value_to_int.keys())) 
    
    plot.savefig("clustermap.png")